In [ ]:
!pip install vaderSentiment

  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)


In [ ]:
!pip install --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]' --user

  Using cached google_cloud_bigquery-3.7.0-py2.py3-none-any.whl (215 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.22.0 requires packaging<22.0.0dev,>=14.3, but you have packaging 23.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from google.cloud import bigquery

In [ ]:
#consulta los registros de la abla reviews_listo
qryReviews = """
Select *
from delorean_dataset.reviews_definitivos_Google
"""
dfReviews = pd.read_gbq(qryReviews, dialect='standard')

In [ ]:
dfReviews[dfReviews['text']!='no data']

,gmap_id,rating,text,text_resp,name,user_id,date,review_id
0,0x89e82aba4127b095:0x13daf60849c1ebcb,1,Bad experience at the restaurant in farmingdal...,,Vonnetta fowler,1.041960e+20,2016-08-11,<NA>
2,0x8640b189325f703d:0x43429c1acfe64b70,1,I drove all the way from Channelview 2 find ou...,,Tracey Ward,1.165127e+20,2020-09-30,<NA>
3,0x89c25de35015c7d5:0x575e1bc92ce14fb5,1,Every. Single. Time. I walk passed this place ...,Maritza I thing you are wrong every one were a...,Maritza Henriquez,1.172840e+20,2020-10-04,<NA>
4,0x89e8313cb957c931:0x8902dd0166ca3a2,1,Translated by Google Tacos do not work here an...,,Isis Bonilla,1.015132e+20,2019-11-21,<NA>
5,0x89c28d4bca2f1bf5:0xeae57872beff50ee,1,The food tastes really bad and the macaroni is...,,Michael Cole,1.027505e+20,2017-07-17,<NA>
...,...,...,...,...,...,...,...,...
1178046,0x80c2cd54e8ecb70f:0xa1783fe819607a9,5,Crimson Coward was excellent. We got the Cri...,"Genevieve,\n\nThank you so much for sharing yo...",GENEVIEVE TORRES,1.140614e+20,2021-04-02,<NA>
1178049,0x88f50f10abda7643:0x7adc202cd6a4e804,5,Delicious tacos and casual dining at affordabl...,,Charlie Hubbard,1.044826e+20,2017-01-21,<NA>
1178051,0x89c25f0708e37dff:0x77fe4c404dbb761,5,Excellent attention. They have music night on ...,,Maria Ocasio,1.029674e+20,2019-07-27,<NA>
1178052,0x809ae54a350445c5:0xcbb4ed28f17dcf1,5,Awesome place for a date night,Thanks! We hope to see you again soon!,The Vincelli's,1.088102e+20,2020-08-23,<NA>


In [ ]:
#instancia el analizador de los sentimientos
sentiment = SentimentIntensityAnalyzer()

In [ ]:
#esta función nos va a ayudar a generar las columnas que necesitamos con la valoración
def pegaSentimientos(texto):
    if texto == 'no data':
        texto={'neg':0,'neu':0,'pos':0,'compound':0}
    registro=dict(sentiment.polarity_scores(texto))
    return pd.Series([registro.get('neg'),registro.get('neu'),registro.get('pos'),registro.get('compound')])


In [ ]:
dfReviews[['neg','neu','pos','compound']]=dfReviews['text'].apply(pegaSentimientos)

In [ ]:
#cuenta el número de palabras
def cuentaPalabras(texto):
    if texto=='no data':
        conteo=0
    else:
        conteo = len(re.findall(r'\w+', texto))
        
    return conteo

In [ ]:
dfReviews['conteo'] = dfReviews['text'].apply(cuentaPalabras)

In [ ]:
dfReviews[dfReviews['text']!='no data'][['text','neg','neu','pos','compound']]

,text,neg,neu,pos,compound
0,All sorts of hard Gristle in the chicken,0.167,0.833,0.000,-0.1027
1,Absolute awful service. And this wasnt the fir...,0.273,0.727,0.000,-0.4588
2,Manager IVEETH I never seen rude manager. Sh...,0.000,0.740,0.260,0.9077
3,Dirty dark unprofessional management,0.756,0.244,0.000,-0.7351
5,I wouldnt bring your family there some of the...,0.097,0.903,0.000,-0.1280
...,...,...,...,...,...
3515531,Nice hospital. Nice doctors,0.000,0.263,0.737,0.6808
3515534,Tiny place big taste Great service too,0.000,0.594,0.406,0.6249
3515536,Quick friendly service. Down home food,0.000,0.610,0.390,0.4939
3515537,Great place for a sandwich or a salad. Great c...,0.000,0.687,0.313,0.8481


In [ ]:
dfReviews.head()

,gmap_id,rating,text,text_resp,name,user_id,date,neg,neu,pos,compound,conteo
0,0x80dd28116a696f1f:0x13ec24a18f0cc3ab,1,All sorts of hard Gristle in the chicken,,dr. MILLR,1.092089e+20,2016-05-09,0.167,0.833,0.00,-0.1027,8
1,0x88f5b287fe59fe1f:0xf5559ac4d396e087,1,Absolute awful service. And this wasnt the fir...,,Janelle Weidman,1.100750e+20,2019-01-12,0.273,0.727,0.00,-0.4588,9
2,0x88f5a6961dae2ef5:0x2eb2cf4ed6ff098b,1,Manager IVEETH I never seen rude manager. Sh...,,eyasu haile,1.176236e+20,2020-10-31,0.000,0.740,0.26,0.9077,43
3,0x808f86a45a4e3c65:0x570f24529932bbe,1,Dirty dark unprofessional management,,Kathy Pavlofsky,1.001334e+20,2017-12-11,0.756,0.244,0.00,-0.7351,4
4,0x864c197e65ae4c8b:0x7306527da929c6cb,1,no data,,Mike Pliley,1.092782e+20,2019-08-24,0.000,1.000,0.00,0.0000,0


In [ ]:
#crea un campo como índice
dfReviews['id'] = np.arange(len(dfReviews))

In [ ]:
dfReviews.to_gbq('delorean_dataset.reviews_definitivos_Google', if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 3144.16it/s]
